### This script will assume that the files you are moving around have the final status
### If you add lower status files to processed_files field, permissions will not work properly

In [ ]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.wfr import *

# get key from keypairs.json
my_key = get_key('')

# Use title to map the correct opf items
# move_titles = ['HiC Processing Pipeline - Preliminary Files']
# move_titles = ["Repli-Seq Pipeline - Preliminary Files"]
move_titles =  []

sets_in_scope = [] # ['4DNACCCC', '4DNACCCCC']

search_url  = '' # /search/?type=...

#run_sets = ff_utils.search_metadata(set_url , key=my_key)
if sets_in_scope:
    run_sets = [ff_utils.get_metadata(i, my_key) for i in sets_in_scope]
elif search_url:
    run_sets = ff_utils.search_metadata(search_url, my_key)

# if True do the action, if false just report
action = False
add_to_existing = True  # if True and there are already proceesed_files will add to them 


# move other processed files to processed files field
def move_opc_to_pc(resp, move_titles, con_key):
    opcolls = resp.get('other_processed_files')
    pc = resp.get('processed_files', [])
    # import pdb; pdb.set_trace()
    # if processed_files field already has values, exit
    if not add_to_existing and pc:
        print('ERROR: There are files in processed_files field, expected empty')
        return False
    # see if there are other_processed_files to move
    if opcolls:
        move_items = []
        for opc in opcolls:
            if opc.get('title') in move_titles:
                move_items.append(opc)
 
        try:
            assert len(move_items) == len(move_titles)
        except:
            print('ERROR: There are more than 1 opf set that has same title')
            return False
        try:
            assert all([mi['type'] == 'preliminary' for mi in move_items])
        except:
            print('ERROR: OPF set status should be preliminary')
            return False
        new_pc = []
        for mi in move_items:
            new_pc.extend(mi.get('files', []))

        new_opc = [i for i in opcolls if i['title'] not in move_titles]
        # Time to patch
        patch_data = {}
        add_on = ""
        #if there is something left in opc, patch it, if not delete field
        if new_opc:
            patch_data['other_processed_files'] = new_opc
        else:
            add_on = 'delete_fields=other_processed_files'
        # patch with processed files 
        # add new pfs to existing pc list (which may be empty if no previous processed_files)
        pc.extend(new_pc)
        patch_data['processed_files'] = pc
        # compare the status
        set_exp_status = resp['status']
        # import pdb; pdb.set_trace()
        for a_file in pc:
            file_resp = ff_utils.get_metadata(a_file, con_key)
            file_status = file_resp['status']
            if file_status != set_exp_status:
                print('ERROR: File status does not match the set, please use release script to update status for all related items')
                return False
        if action:
            res = ff_utils.patch_metadata(patch_data, resp['uuid'], key = con_key, add_on = add_on)
            print(len(new_pc), 'files moved to pf')
            print(res.get('status'))
        else:
            print(len(new_pc), 'files will move to pf')
            print(patch_data)
            if add_on:
                print(add_on)
        return True
    else:
        return False



set_w_apf = 0
exp_w_apf = 0
counter = 0

if not action:
    print('\nTHIS IS JUST A TEST\n')   
        
print(len(run_sets), 'experiment sets in scope')
        
for a_set in run_sets:
    set_resp = ff_utils.get_metadata(a_set['uuid'],key=my_key, add_on='frame=raw')
    counter += 1
    print(counter, set_resp['accession'])
    exps = set_resp['experiments_in_set']
    res =  move_opc_to_pc(set_resp, move_titles, my_key)
    if res:
        set_w_apf += 1
    for exp in exps:
        exp_resp = ff_utils.get_metadata(exp, key=my_key, add_on='frame=raw')
        res_e =  move_opc_to_pc(exp_resp, move_titles, my_key)
        if res_e:
            exp_w_apf += 1
    print()

print(set_w_apf, 'sets are updated')
print(exp_w_apf, 'exps are updated')